<a href="https://colab.research.google.com/github/hun9008/ML_TeamProject_24SS/blob/main/PreTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [41]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Physical devices cannot be modified after being initialized


In [1]:
!pip install -q keras

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt

In [3]:
import os
os.chdir("/content/drive/My Drive/ML_TeamProject/preprocessed_images_40")
!ls

mild  normal  severe  TEST_DIR	TRAIN_DIR


In [4]:
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

#Augumentation with only zoom and grayscale format

train_datagen2 = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip = True,
                             zoom_range = 0.3
                            )

#Corresponding generator
train_generator2 = train_datagen2.flow_from_directory(
    train,
    color_mode='grayscale',
    batch_size=32,
    class_mode = 'categorical',
    target_size=(224,224),
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen2 = ImageDataGenerator()
#Using testing data as validation data

validation_generator2 = test_datagen2.flow_from_directory(
    test,
    color_mode='grayscale',
    batch_size=32,
    class_mode = 'categorical',
    target_size=(224,224),
    seed=42 )

x_train,y_train = next(train_generator2)
x_test,y_test = next(validation_generator2)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 224, 224, 1)
(32, 224, 224, 1)


# Basic CNN

In [5]:
#Basic CNN Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
import numpy as np
from keras.callbacks import ModelCheckpoint

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),padding='same', activation='relu', input_shape = (224, 224, 1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [7]:
#for fitting the model
from keras.callbacks import EarlyStopping

stop = EarlyStopping(
    monitor = 'val_accuracy',
    mode='max',
    patience= 8,
    verbose = 2,
    restore_best_weights = True
)

STEPS = train_generator2.samples//train_generator2.batch_size
VAL_STEPS = validation_generator2.samples//validation_generator2.batch_size
EPOCHS = 20

results = model.fit(
train_generator2,
steps_per_epoch=STEPS,
epochs=EPOCHS,
validation_data=validation_generator2,
validation_steps=VAL_STEPS,
callbacks=[stop]
)

Epoch 1/20
12/12 [==============================] - 126s 10s/step - loss: 13101.2637 - accuracy: 0.3280 - val_loss: 5666.2671 - val_accuracy: 0.4141
Epoch 2/20
12/12 [==============================] - 3s 291ms/step - loss: 3322.5381 - accuracy: 0.4635 - val_loss: 343.1749 - val_accuracy: 0.5703
Epoch 3/20
12/12 [==============================] - 3s 219ms/step - loss: 638.3936 - accuracy: 0.5344 - val_loss: 529.0707 - val_accuracy: 0.5234
Epoch 4/20
12/12 [==============================] - 4s 299ms/step - loss: 300.0597 - accuracy: 0.6667 - val_loss: 581.9482 - val_accuracy: 0.5781
Epoch 5/20
12/12 [==============================] - 3s 282ms/step - loss: 146.0686 - accuracy: 0.7328 - val_loss: 294.9182 - val_accuracy: 0.7188
Epoch 6/20
12/12 [==============================] - 3s 217ms/step - loss: 112.3003 - accuracy: 0.7725 - val_loss: 161.0994 - val_accuracy: 0.6641
Epoch 7/20
12/12 [==============================] - 3s 223ms/step - loss: 60.8739 - accuracy: 0.7963 - val_loss: 103.406

# ResNet50

In [8]:
from keras import applications
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import SGD, Adam

In [11]:
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (224,224,1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation= 'softmax')(x)
resnetmodel = Model(inputs = base_model.input, outputs = predictions)
adam = Adam(learning_rate=0.001)
resnetmodel.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
EPOCH = 30
stopping_resNet = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5,
    verbose = 2,
    restore_best_weights = True
)

restNetModel = resnetmodel.fit(
    train_generator2,
    steps_per_epoch=STEPS,
    epochs=EPOCH,
    validation_data=validation_generator2,
    validation_steps=VAL_STEPS,
    callbacks=[stopping_resNet]
)

Epoch 1/30
12/12 [==============================] - 39s 882ms/step - loss: 2.5068 - accuracy: 0.4101 - val_loss: 2104.5269 - val_accuracy: 0.2969
Epoch 2/30
12/12 [==============================] - 5s 366ms/step - loss: 1.0140 - accuracy: 0.6615 - val_loss: 514.8922 - val_accuracy: 0.2969
Epoch 3/30
12/12 [==============================] - 4s 364ms/step - loss: 0.8652 - accuracy: 0.7275 - val_loss: 170.7675 - val_accuracy: 0.2969
Epoch 4/30
12/12 [==============================] - 5s 371ms/step - loss: 0.6128 - accuracy: 0.7910 - val_loss: 14.5396 - val_accuracy: 0.3047
Epoch 5/30
12/12 [==============================] - 4s 361ms/step - loss: 0.5961 - accuracy: 0.8228 - val_loss: 3.0730 - val_accuracy: 0.3047
Epoch 6/30
12/12 [==============================] - 5s 374ms/step - loss: 0.3926 - accuracy: 0.8466 - val_loss: 2.6105 - val_accuracy: 0.4141
Epoch 7/30
12/12 [==============================] - 5s 374ms/step - loss: 0.3831 - accuracy: 0.8915 - val_loss: 1.8202 - val_accuracy: 0.43

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 224, 224, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 1605632)           0         
                                                                 
 dense (Dense)               (None, 100)               160563300 
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 160563923 (612.50 MB)
Trainable params: 160563923 (612.50 MB)
Non-trainable params: 0 (0.00 Byte)
__________

In [33]:
resnetmodel.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 1)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         3200      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [37]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 3)
(32, 3)


In [38]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [40]:
# prompt: 위에서 만든 CNN과 ResNet에 대해 test로 accuracy를 측정

# Basic CNN
test_loss, test_acc = model.evaluate(x_test, y_test)
print('==== CNN ====')
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)


InternalError: Graph execution error:

Detected at node sequential/conv2d/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/di